# Pygame

## Uvod


### Instalacija

In [1]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.


### Osnove

In [2]:
# importujemo i inicijalizujemo pygame biblioteku
import pygame
pygame.init()

# Podešavamo prostor po kome ćemo crtati
display = pygame.display.set_mode((500, 500))

# Konstane koje sadrže neke od boja
CRNA = pygame.Color("black")
PINK = pygame.Color(230, 70, 230)

# Glavna petlja 
game_over = False
while not game_over:
    # Dohvata listu svih event-ova koji su se dogodili (pritiskanje tastature, miša, džojstika...)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            game_over = True
    
    # Ažurirarnje frejma
    ####################
    # Puni prozor crnom bojom
    display.fill(CRNA)
    # Crtamo beli krug na koordinatama 250, 250 sa poluprečnikom 30
    pygame.draw.circle(display, PINK, (250, 250), 30)
    ####################
    
    # Flipovanje displeja (prikazujemo frejm koji smo napravili do ovog trenutka)
    pygame.display.flip()


# Čisti display i gasi prozor
pygame.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Koncepti

#### Display i Surface (displaj i povrsi)

Surface - pravi jedan pravougaonik na kom možete da crtate, ili na koji možete da stavite neku sliku.
<br>Display - ili prozor ili fullscreen, sve što se prikazuje na njemu će krajni korisnik videti, on se kreira se medotodm .set_mode() i mi na ovu površ šaljemo sve funkcije koje će nam prikazati neki oblik, npr krug

## Igrica

### Biblioteke i globalne promenljive

In [3]:
# Importujemo pygame
import pygame
import random

# Importujemo i moduo koji zna koje dugme je pritisnuto
from pygame.locals import *


# Inicijalizujemo pygame biblioteku
pygame.init()

# Definišemo konstane koje će se koristiti u programu
EKRAN_SIRINA = 800
EKRAN_VISINA = 600
# Boje
CRNA = pygame.Color("black")
BELA = pygame.Color("white")
PINK = pygame.Color(230, 70, 230)

### Definišemo klase

In [4]:
# Definišemo klasu Igrač kao podklasu klase pygame.sprite.Sprite (Sprite)
# Površina koja će se crtati na ekranu će biti i imati sva svojstva klase "Igrac"
class Igrac(pygame.sprite.Sprite):
    # Svojstva
    
    # Konstruktor
    def __init__(self):
        # Pozivamo konstruktor za Sprite klasu od koje nasleđujemo sve atribute 
        pygame.sprite.Sprite.__init__(self)
        
        # Kreirali smo sliku bloka i popunili smo je belom bojom, to je mogla biti i slika
        self.povrs = pygame.Surface( (25, 60) )
        self.povrs.fill(BELA)
        
        # Pravi objekat pravougaonika po dimenzijama površi koju smo napravili iznad
        self.pravougaonik = self.povrs.get_rect()
    
    # Metode
    # Metoda za pomeranje sprite-a u zavisnosti od pritisnutog dugmeta
    def update(self, dugmici):
        if dugmici[K_UP]:
            self.pravougaonik.move_ip(0, -5)
        if dugmici[K_DOWN]:
            self.pravougaonik.move_ip(0, 5)
        if dugmici[K_LEFT]:
            self.pravougaonik.move_ip(-5, 0)
        if dugmici[K_RIGHT]:
            self.pravougaonik.move_ip(5, 0)
        
        # Pravimo proveru da li je igrač izašao sa ekrana
        # left - x, top - y
        if self.pravougaonik.top <= 0:
            self.pravougaonik.top = 0
        if self.pravougaonik.bottom >= EKRAN_VISINA:
            self.pravougaonik.bottom = EKRAN_VISINA
        if self.pravougaonik.left <= 0:
            self.pravougaonik.left = 0
        if self.pravougaonik.right >= EKRAN_SIRINA:
            self.pravougaonik.right = EKRAN_SIRINA
            

In [5]:
class Neprijatelj(pygame.sprite.Sprite):
    # Svojstva
    # Konstruktor
    def __init__(self):
        # Pozivamo konstruktor za Sprite klasu od koje nasleđujemo sve atribute 
        pygame.sprite.Sprite.__init__(self)
        
        # Kreirali smo sliku bloka i popunili smo je belom bojom, to je mogla biti i slika
        self.povrs = pygame.Surface([10, 20])
        self.povrs.fill(BELA)
        # Generisemo nasumicno poziciju neprijatelja po x i y osi
        # Pravi objekat pravougaonika po dimenzijama površi koju smo napravili iznad
        centar_pravougaonika = (random.randint(0, EKRAN_SIRINA), random.randint(-100, 0))
        self.pravougaonik = self.povrs.get_rect(center=centar_pravougaonika)
        # Postavljamo brzinu kretanja neprijatelja od gore ka dole
        self.brzina = random.randint(3, 20)
        
        
    # Metode
    def update(self):
        # Pomeramo neprijatelja za brzinu piksela na dole
        self.pravougaonik.move_ip(0, self.brzina)
        # Ako je presao donju ivicu ekrana, uklanjamo ga
        if self.pravougaonik.top > EKRAN_VISINA:
            self.kill()

### Glavno telo programa

In [6]:

# Pravimo ekran na kome ćemo iscrtavati sve (display)
display = pygame.display.set_mode((EKRAN_SIRINA, EKRAN_VISINA))

# Pravimo naš event za dodavanje neprijatelja u grupu
DODAJMETEOR = pygame.USEREVENT + 1
pygame.time.set_timer(DODAJMETEOR, 100)

# Pravimo igrača (neki pravougaonik, za sada)
igrac = Igrac()

# Grupa neprijatelji sadrži sve elemente klase Neprijatelji 
neprijatelji = pygame.sprite.Group()
# Pravimo grupu u kojoj će se nalaziti svi sprite elementi, radi lakšeg iscrtavanja/ažuriranja, dodajemo igrača u grupu(listu)
svi_sprite_elementi = pygame.sprite.Group()
svi_sprite_elementi.add(igrac)

# Glavna petlja (jedna iteracija == jedan frejm)
# 1. Obrađujemo ulaze korisnika
# 2. Ažuriramo stanja svih objekata
# 3. Puštanje tog frejma, puštamo muzičke efekte
# 4. Limitiramo brzinu izvršavanja petlje (limitiramo broj frejmova u sekundi)
game_over = False
while not game_over:
    # 1. Obrađujemo ulaze korisnika
    # Dohvata listu svih event-ova koji su se dogodili (pritiskanje tastature, miša, džojstika...)
    for event in pygame.event.get():
        # X dugme na prozoru
        if event.type == pygame.QUIT:
            game_over = True
        # Bilo koje dugme na tastaturi
        elif event.type == KEYDOWN:
            # ESC
            if event.key == K_ESCAPE:
                game_over = True
        # Ubacivanje neprijatelja
        elif event.type == DODAJMETEOR:
            # Pravimo neprijatelja, dodajemo ga na odgovarajuce grupe
            meteor = Neprijatelj()
            neprijatelji.add(meteor)
            svi_sprite_elementi.add(meteor)
            
    
    # Dohvata rečnik stanja svih dugmića na tastaturi (True/False, pritisnut/nije pritisnut)
    pritisnuti_dugmici = pygame.key.get_pressed()
    #######################################            
    
    # 2. Ažuriramo stanja svih objekata
    display.fill(CRNA)
    
    # Ažuriramo stanje(mesto) igrača u odnosu na pritisnute dugmiće
    igrac.update(pritisnuti_dugmici)
    
    # Ažuriramo poziciju neprijatelja
    neprijatelji.update()
    
    # Postavljamo igrača na display
    # blit() - Block Transfer, preslikava jednu površinu na drugu
    # display.blit(igrac_brod.povrs, (EKRAN_SIRINA/2, EKRAN_VISINA/2))
    
    # Pomeranjem koordinata pravougaonika crta nam se površ od igrača
    # display.blit(igrac.povrs, igrac.pravougaonik)
    
    # Crtamo grupu svih sprite elemenata od jednom
    for element in svi_sprite_elementi:
        display.blit(element.povrs, element.pravougaonik)
    
    #######################################
    
    # 3. Puštanje tog frejma, puštamo muzičke efekte
    # Flipovanje displeja (prikazujemo frejm koji smo napravili do ovog trenutka)
    pygame.display.flip()
    
    
# Čisti display i gasi prozor
pygame.quit()